Importing dependencies and loading csv file

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
ds=pd.read_csv("/content/drive/MyDrive/ML Projects/Twitter_live_Sentiment_Analysis/sentiment140.csv", encoding = "ISO-8859-1")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   target  1600000 non-null  int64 
 1   id"     1600000 non-null  int64 
 2   date    1600000 non-null  object
 3   flag    1600000 non-null  object
 4   user    1600000 non-null  object
 5   text    1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


In [ ]:
ds.drop(columns=['id"','date','flag','user'],inplace=True)


In [ ]:
y=ds['target']

In [ ]:
ds.head()


,target,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [ ]:
ds.duplicated().sum() #checking for duplicates

16309

In [ ]:
ds=ds.drop_duplicates(keep='first')

Target value for all tweets is 0 or 4 at which implies negative and positive

In [ ]:
ds = ds.sample(frac = 1)#shuffling

In [ ]:
ds.head()

,target,text
518205,0,@xohanna no we haven't got it. my auntie has ...
101795,0,WORK time
133310,0,Don't wanna be here
954367,4,Its past 3 in the morning so I'm going to bed ...
164286,0,i want a dog...


In [ ]:
ds_split=np.array_split(ds,40)#splitting 1.6 Million tweets into 40 equal parts

In [ ]:
ds_split[0]['target'].value_counts()

0    19881
4    19712
Name: target, dtype: int64

In [ ]:
df=ds_split[0] #df stores the fraction of our original dataframe
df.head()

,target,text
518205,0,@xohanna no we haven't got it. my auntie has ...
101795,0,WORK time
133310,0,Don't wanna be here
954367,4,Its past 3 in the morning so I'm going to bed ...
164286,0,i want a dog...


In [ ]:
df=df.reset_index()

In [ ]:
df.drop(columns=['index'],inplace=True)

In [ ]:
df.head()

,target,text
0,0,@xohanna no we haven't got it. my auntie has ...
1,0,WORK time
2,0,Don't wanna be here
3,4,Its past 3 in the morning so I'm going to bed ...
4,0,i want a dog...


In [ ]:
df['target'].value_counts()

0    19881
4    19712
Name: target, dtype: int64

Preprocessing

In [ ]:
!pip install nltk


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import nltk

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer

In [ ]:
from nltk.stem.porter import PorterStemmer

In [ ]:
import re
import string

In [ ]:

def pp(text):

  text=re.sub(r'https?:\/\/.*[\r\n]*','', text)
  text=re.sub(r'#','',text)

  tokenizer = TweetTokenizer(preserve_case=False,strip_handles=True, reduce_len=True)
  text= tokenizer.tokenize(text)

  y=[]
  for i in text:
    if i.isalnum():
      y.append(i)

  text=y[:]
  y.clear()

  stop_words = set(stopwords.words('english'))
  punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
  for i in text:
    if i not in stop_words and i not in punc:
      y.append(i)

  text=y[:]
  y.clear()

  ps=PorterStemmer()
  for i in text:
    y.append(ps.stem(i))

  return " ".join(y)


In [ ]:
pp(df['text'][1])

'work time'

In [ ]:
df['text'][1]

'WORK time '

In [ ]:

df['pp_text']=df[df.columns[1]].apply(pp)

In [ ]:
df.head()

,target,text,pp_text
0,0,@xohanna no we haven't got it. my auntie has ...,got aunti though might go watch lmao
1,0,WORK time,work time
2,0,Don't wanna be here,wanna
3,4,Its past 3 in the morning so I'm going to bed ...,past 3 morn go bed goodnight sleep well
4,0,i want a dog...,want dog


Dropping empty rows

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39593 entries, 0 to 39592
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   target   39593 non-null  int64 
 1   text     39593 non-null  object
 2   pp_text  39593 non-null  object
dtypes: int64(1), object(2)
memory usage: 928.1+ KB


Model

In [ ]:
y=df['target'].values

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf= TfidfVectorizer(max_features=10000)
X=tfidf.fit_transform(df['pp_text']).toarray()
len(tfidf.vocabulary_)

10000

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size =0.3,
                                                    random_state=1)


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()
mnb.fit(X_train, y_train)
y_pred=mnb.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.7274793736319246


In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(n_estimators=100,criterion='entropy',random_state=0)
rf=rf.fit(X_train,y_train)
y_pred3 = rf.predict(X_test)


In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred3))

0.7296682943256441


In [ ]:
import pickle
pickle.dump(tfidf,open('sentiment_analysis_vectorizer.pkl','wb'))
pickle.dump(mnb,open('sentiment_analysis_model.pkl','wb'))
